Install neccessary packages 

In [1]:
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [1]:
from urllib import request as rq
import pandas as pd
import os
import pyarrow as pa # not sure will i use this 
import numpy as np
import ipywidgets
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, VBox
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder

Download the Dublin Bus data from https://data.gov.ie/organization/dublin-city-council?tags=Transport+and+Infrastructure

In [3]:
# Check folder existence before downloading
if os.path.exists('./Zips/Bus'):
    print("Zips folder exists")
else:
    os.makedirs('./Zips/Bus/', exist_ok = True)

url = "https://opendata.dublincity.ie/TrafficOpenData/sir010113-310113.zip"
busFile = rq.urlretrieve(url, './Zips/Bus/DublinBusdata.zip' )    

Zips folder exists


In [4]:
import zipfile as zip
# Zip creates its own folders - no need to check for folder existence
with zip.ZipFile(busFile[0],  mode='r') as arc: 
    arc.extractall('./Data/Bus/Gz/')

Use a generator to load the data into pandas data frame https://pandasninja.com/2019/04/how-to-read-lots-of-csv-files-easily-into-pandas/#:~:text=How%20to%20read%20lots%20of%20csv%20files%20easily,need%20...%204%204.%20Leverage%20regular%20expressions%20


In [28]:

def load_Files(files):
    columns = ['Timestamp', 'LineID', 'Direction', 'JourneyPatternID', 'TimeFrame', 'VehicleJourneyID', 'Operator', 'Congestion', 'LonWGS84', 'LatWGS84', 'Delay', 'BlockID', 'VehicleID', 'StopID', 'AtStop']
    for f in files:
        print(f)
        yield pd.read_csv('./Data/Bus/Gz/' + f, compression='gzip', delimiter=',', header=0, names=columns, parse_dates=True, low_memory=True)

files = os.listdir('./Data/Bus/Gz/')
DBfiles = [f for f in files if f.endswith('.gz')]

if os.path.exists('./Data/Bus/CleanedBusData.parquet'):
    df = pd.read_parquet('./Data/Bus/CleanedBusData.parquet')
else:
    df = pd.concat(load_Files(DBfiles))

Download the weather station details https://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv

In [29]:
# Getting the list of categorical columns
CategoricalColumns = list(set(list(df.select_dtypes(exclude=[np.number]).columns)))
NumericalColumns = list(set(list(df.select_dtypes([np.number]).columns)))

print(CategoricalColumns)
print(NumericalColumns)

ColumnsToExclude = ['JourneyPatternID', 'StopID']
CategoricalColumns = list(set(CategoricalColumns) - set(ColumnsToExclude))
NumericalColumns = list(set(NumericalColumns)-set(ColumnsToExclude))




['TimeFrame', 'Operator']
['VehicleJourneyID', 'Timestamp', 'Direction', 'LatWGS84', 'Delay', 'BlockID', 'Congestion', 'VehicleID', 'AtStop', 'LineID', 'LonWGS84']


In [ ]:
df = df.drop(ColumnsToExclude, axis=1)
df = df.dropna()


In [30]:
df.head()

,Timestamp,LineID,Direction,TimeFrame,VehicleJourneyID,Operator,Congestion,LonWGS84,LatWGS84,Delay,BlockID,VehicleID,AtStop
0,1356998405000000,27.0,0,2012-12-31,3883,RD,0,-6.233417,53.342232,0,27017,33521,0
1,1356998407000000,40.0,0,2012-12-31,2226,HN,0,-6.278250,53.416683,0,40206,33142,0
2,1356998407000000,7.0,0,2012-12-31,6106,D1,0,-6.231633,53.317768,0,7019,43004,1
3,1356998411000000,747.0,0,2012-12-31,3531,SL,0,-6.254617,53.355484,-454,747007,40039,0
4,1356998411000000,56.0,0,2012-12-31,1830,RD,0,-6.233183,53.342201,0,56001,33488,0


In [31]:
df['datetime'] = pd.to_datetime(df['Timestamp'], unit='us')

In [32]:
df.head

<bound method NDFrame.head of                Timestamp  LineID  Direction   TimeFrame  VehicleJourneyID  \
0       1356998405000000    27.0          0  2012-12-31              3883   
1       1356998407000000    40.0          0  2012-12-31              2226   
2       1356998407000000     7.0          0  2012-12-31              6106   
3       1356998411000000   747.0          0  2012-12-31              3531   
4       1356998411000000    56.0          0  2012-12-31              1830   
...                  ...     ...        ...         ...               ...   
603915  1359632694000000    39.0          0  2013-01-31              3247   
603916  1359632694000000     9.0          0  2013-01-31              3649   
603917  1359632694000000    41.0          0  2013-01-31               112   
603918  1359632694000000   145.0          0  2013-01-31              6810   
603919  1359632694000000   145.0          0  2013-01-31              6471   

       Operator  Congestion  LonWGS84   LatWG

In [33]:
print ('The data has {} Rows and {} columns'.format(df.shape[0],df.shape[1]))
print("The types of columns are:")
df.dtypes

The data has 44453044 Rows and 14 columns
The types of columns are:


Timestamp                    int64
LineID                     float64
Direction                    int64
TimeFrame                   object
VehicleJourneyID             int64
Operator                    object
Congestion                   int64
LonWGS84                   float64
LatWGS84                   float64
Delay                        int64
BlockID                      int64
VehicleID                    int64
AtStop                       int64
datetime            datetime64[ns]
dtype: object

In [36]:
def num_missing(x):
    return len(x.index)-x.count()

def num_unique(x):
    return len(np.unique(x))

temp_df = df.describe().T
missing_df = pd.DataFrame(df.apply(num_missing, axis=0)) 
missing_df.columns = ['missing']
unq_df = pd.DataFrame(df.apply(num_unique, axis=0))
unq_df.columns = ['unique']
types_df = pd.DataFrame(df.dtypes)
types_df.columns = ['DataType']

In [42]:
df['datetime'].unique()

array(['2013-01-01T00:00:05.000000000', '2013-01-01T00:00:07.000000000',
       '2013-01-01T00:00:11.000000000', ...,
       '2013-01-31T11:44:29.000000000', '2013-01-31T11:44:40.000000000',
       '2013-01-31T11:44:54.000000000'], dtype='datetime64[ns]')

In [38]:
summary_df = temp_df.join(missing_df).join(unq_df).join(types_df)
summary_df

,count,mean,std,min,25%,50%,75%,max,missing,unique,DataType
Timestamp,44453044.0,1.358318e+15,7.394966e+11,1.356998e+15,1.357679e+15,1.358327e+15,1.358955e+15,1.359633e+15,0,813102,int64
LineID,44453044.0,7.796879e+01,1.146419e+02,1.000000e+00,2.500000e+01,4.000000e+01,8.300000e+01,7.470000e+02,0,66,float64
Direction,44453044.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,1,int64
VehicleJourneyID,44453044.0,9.417338e+03,6.159986e+04,1.000000e+00,2.536000e+03,4.718000e+03,6.769000e+03,9.998560e+05,0,18614,int64
Congestion,44453044.0,1.163315e-02,1.072279e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0,2,int64
LonWGS84,44453044.0,-6.272804e+00,8.389086e-02,-6.617517e+00,-6.308651e+00,-6.261600e+00,-6.233166e+00,-6.052917e+00,0,456755,float64
LatWGS84,44453044.0,5.334513e+01,5.488335e-02,5.306802e+01,5.332005e+01,5.334645e+01,5.337532e+01,5.360873e+01,0,131327,float64
Delay,44453044.0,-2.816643e+01,4.729787e+02,-1.504500e+04,-2.090000e+02,0.000000e+00,1.040000e+02,1.161220e+05,0,12638,int64
BlockID,44453044.0,1.092380e+05,1.921183e+05,3.900000e+02,1.602000e+04,4.020500e+04,8.400400e+04,8.350020e+05,0,1228,int64
VehicleID,44453044.0,3.542929e+04,3.281127e+03,2.804700e+04,3.330800e+04,3.352500e+04,3.802500e+04,4.307800e+04,0,911,int64


In [43]:
col_names = list(types_df.index)
num_cols = len(col_names)
index = range(num_cols)
cat_index = []
for i in index:
    if col_names[i] in CategoricalColumns:
        cat_index.append(i)
summary_df_cat = missing_df.join(unq_df).join(types_df.iloc[cat_index], how='inner') #Only summarize categorical columns
summary_df_cat

,missing,unique,DataType
TimeFrame,0,32,object
Operator,0,8,object


In [44]:
df.to_parquet('./Data/Bus/CleanedBusData.parquet')

In [45]:
url = 'https://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv'


Load the weather statio details for Dublin

In [46]:
weatherStations = pd.read_csv(url)
weatherStations = weatherStations['county'].str.contains('Dublin')
weatherStations.head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: county, dtype: bool

,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0,Antrim,5880,LH_RATHLIN_WEST,10,309200,451800,55.30083,-6.28028,2000,(null)
1,Carlow,4415,TULLOW (Waterworks),76,284700,173400,52.80528,-6.74306,1985,(null)
2,Carlow,2414,BORRIS G.S.,85,272400,150700,52.60278,-6.93056,1944,1991
3,Carlow,1214,CARLOW (SUGAR FACTORY),58,272200,178400,52.85139,-6.92778,1953,1960
4,Carlow,115,HACKETSTOWN RECTORY,182,297600,180500,52.86667,-6.55000,1910,1944
